In [1]:
using SummationByParts, LinearAlgebra
using SummationByParts.Cubature, SummationByParts.SymCubatures
using Latexify

[ Info: Precompiling SummationByParts [55d25451-638a-4804-8089-59924bcb4a40]
[ Info: Skipping precompilation since __precompile__(false). Importing SummationByParts [55d25451-638a-4804-8089-59924bcb4a40].


WebIO._IJuliaInit()

# Construct SBP Operators

Okay, we have derived some existing or new quadrature rules. How do we proceed to construct SBP operators?

## SBP Operators on Triangles (with LG or LGL facet nodes)

Consider the quadrature rule we derived for SBP diagonal-E operators.

In [2]:
qf = 11; # the facet quadrature must be qf=q+mod(q,2)+1
cub,_ = SummationByParts.Cubature.quadrature(qf, internal=false); # get LGL nodes for the facet nodes
xedge = cub.params;

In [3]:
cub_tri, vtx_tri = SummationByParts.deriveTriCubatureDiagE(q=9,
                                                    vertices=true, 
                                                    midedges=true, 
                                                    numS21=3, 
                                                    numedge=2, 
                                                    numS111=1, 
                                                    centroid=false,
                                                    delta1=1e-2, delta2=1e-2, xinit = [], xedge = xedge, verbose=false)

----------------------------------------------------------------------------------------------
iter_pso = 22400:  iter_lma = 5624:  nperturb_pso = 80:  res norm = 1.8428348086265596e-15
----------------------------------------------------------------------------------------------


(TriSymCub{Float64}(7, 8, 33, true, true, false, 2, 3, 1, [0, 5, 3], [0.18756071259743237, 0.9134518938842776, 0.5163553987937517, 0.9151119481392835, 0.7344243967353571, 0.5426661668128369, 0.1601142330691843], [0.0021943470879656466, 0.02335234785435418, 0.08773819266871097, 0.10176391730890032, 0.1795919549730242, 0.015061783552305814, 0.019549042149360124, 0.10140212768518968]), [-1.0 -1.0; 1.0 -1.0; -1.0 1.0])

To build SBP diagonal-E opertor using the above quadrature, we proceed as follows:

In [4]:
w, Q, E = SummationByParts.buildoperators(cub_tri, vtx_tri, 5, vertices=true)

([0.0021943470879656466, 0.0021943470879656466, 0.0021943470879656466, 0.02335234785435418, 0.02335234785435418, 0.02335234785435418, 0.08773819266871097, 0.08773819266871097, 0.08773819266871097, 0.10176391730890032  …  0.019549042149360124, 0.019549042149360124, 0.019549042149360124, 0.019549042149360124, 0.10140212768518968, 0.10140212768518968, 0.10140212768518968, 0.10140212768518968, 0.10140212768518968, 0.10140212768518968], [-0.023809523809523794 -0.0017805706134543075 … 0.00981478625894704 -0.003249346571973458; 0.0017805706134543075 0.02380952380952383 … 0.028468751179452738 0.02572428221626448; … ; -0.00981478625894704 -0.028468751179452738 … 0.0 -0.07712692336468223; 0.003249346571973458 -0.02572428221626448 … 0.07712692336468223 0.0;;; -0.02380952380952383 -0.0008902853067271915 … 0.010066170017343412 -0.004621462680110402; 0.0008902853067270952 3.469446951953614e-17 … 0.002744468963188216 -0.0027444689631881325; … ; -0.010066170017343412 -0.002744468963188216 … 0.0 -0.154

Now we can access all operators as follows:

In [5]:
Qx = Q[:,:,1];
Qy = Q[:,:,2];
H = diagm(w);
Ex = E[:,:,1];
Ey = E[:,:,2];
Dx = inv(H)*Qx;
Dy = inv(H)*Qy;
Sx = Qx - 0.5.*Ex;
Sy = Qy - 0.5.*Ey;

We can check if the SBP property is satisfied.

In [6]:
norm(Qx+Qx' - Ex)

1.4642778842076102e-17

In [7]:
norm(Qy+Qy' - Ey)

1.4642778842076102e-17

## SBP Operators on Tetrahedra

Consider the quadrature rule of degree 6 that we have derived previously to construct a degree $p=3$ SBP diagonal-E operator. 

In [8]:
q=6;
p=3;
qf=2*p;

In [9]:
cub_facet, vtx_facet = SummationByParts.deriveTriCubatureGamma(q=qf,
                                                       vertices=true,
                                                       midedges=false,
                                                       numS21=2,
                                                       numedge=1,
                                                       numS111=0,
                                                       centroid=false,
                                                       xinit=[],
                                                       delta1=1e-3,delta2=1e-1,verbose=false)

----------------------------------------------------------------------------------------------
iter_pso = 2400:  iter_lma = 620:  nperturb_pso = 32:  res norm = 3.9585412909313905e-14
----------------------------------------------------------------------------------------------


(TriSymCub{Float64}(3, 4, 15, true, false, false, 1, 2, 0, [0, 3, 1], [0.8506802519794946, 0.2372273727931829, 0.6922540583740098], [0.014260718614410178, 0.330358977291136, 0.20376930605390753, 0.05913883235361154]), [-1.0 -1.0; 1.0 -1.0; -1.0 1.0])

In [10]:
xedge = cub_facet.params
cub_tet, vtx_tet = SummationByParts.deriveTetCubatureDiagE(q=q,
                                                   vertices=true,
                                                   numS31=1,
                                                   midedges=false, 
                                                   numS22=1,
                                                   numfaceS21=2, 
                                                   numedge=1, 
                                                   numS211=0,
                                                   numfaceS111=0, 
                                                   facecentroid=false,
                                                   numS1111=0,
                                                   centroid=true,
                                                   xinit=[],
                                                   xedge=xedge,
                                                   delta1=1e-3,
                                                   delta2=1e-1,
                                                   verbose=false)

----------------------------------------------------------------------------------------------
iter_pso = 0:  iter_lma = 24:  nperturb_pso = 0:  res norm = 2.5932967513921142e-15
----------------------------------------------------------------------------------------------


(TetSymCub{Float64}(5, 7, 51, true, false, true, false, 1, 2, 0, 1, 1, 0, 0, [1, 2, 1, 3, 0], [0.3931484668771844, 0.257863918563195, 0.8506802519794946, 0.2372273727931829, 0.6922540583740098], [0.000831199213836366, 0.06707451628396921, 0.08700624630786465, 0.025281032406422304, 0.014487003956737961, 0.004650827385021457, 0.006646628516743191]), [-1.0 -1.0 -1.0; 1.0 -1.0 -1.0; -1.0 1.0 -1.0; -1.0 -1.0 1.0])

Now, we can simply construct the an SBP operator of degree $p=3$ as follows:

In [11]:
# We must provide the facet operator type, currently the availabe types are, :Omega and :DiagE
# To enforce using the facet quadrature we derived, we must also provide facetcub and facetvtx; 
# otherwise, available default facet operators saved in the code will be used
w, Q, E = SummationByParts.buildoperators(cub_tet, vtx_tet, p, faceopertype=:DiagE, facecub=cub_facet, facevtx=vtx_facet)

([0.000831199213836366, 0.000831199213836366, 0.000831199213836366, 0.000831199213836366, 0.06707451628396921, 0.06707451628396921, 0.06707451628396921, 0.06707451628396921, 0.08700624630786465, 0.08700624630786465  …  0.004650827385021457, 0.004650827385021457, 0.004650827385021457, 0.004650827385021457, 0.004650827385021457, 0.004650827385021457, 0.004650827385021457, 0.004650827385021457, 0.004650827385021457, 0.006646628516743191], [-0.007130359307205089 -0.0011963999592268508 … -5.298891963690636e-5 0.00326415161681456; 0.0011963999592268508 0.007130359307205089 … 5.2988919636896937e-5 -0.003264151616814594; … ; 5.298891963690636e-5 -5.2988919636896937e-5 … 0.0 -3.120913400412216e-17; -0.00326415161681456 0.003264151616814594 … 3.120913400412216e-17 0.0;;; -0.007130359307205089 -0.0005981999796134143 … -0.002704982226496651 0.003264151616814516; 0.0005981999796134143 0.0 … -0.0026519933068597464 -5.319327296409045e-17; … ; 0.002704982226496651 0.0026519933068597464 … 0.02956941617

Then, we access all the remaining operators as:

In [12]:
Qx = Q[:,:,1];
Qy = Q[:,:,2];
Qz = Q[:,:,3];
H = diagm(w);
Ex = E[:,:,1];
Ey = E[:,:,2];
Ez = E[:,:,3];
Dx = inv(H)*Qx;
Dy = inv(H)*Qy;
Dz = inv(H)*Qz;
Sx = Qx - 0.5.*Ex;
Sy = Qy - 0.5.*Ey;
Sz = Qz - 0.5.*Ez;

Check if the SBP property is satisfied:

In [13]:
norm(Qx+Qx' - Ex)

0.0

In [14]:
norm(Qy+Qy' - Ey)

0.0

In [15]:
norm(Qz+Qz' - Ez)

0.0